In [1]:
from common import *
from structures import *

DAY = 17

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 17: Clumsy Crucible ---</h2><p>The lava starts flowing rapidly once the Lava Production Facility is operational. As you <span title="see you soon?">leave</span>, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.</p>
<p>As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!</p>
<p>You land near the gradually-filling pool of lava at the base of your new <em>lavafall</em>. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large <a href="https://en.wikipedia.org/wiki/Crucible" target="_blank">crucibles</a> on wheels.</p>
<p>The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.</p>
<p>To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible <em>from the lava pool to the machine parts factory</em>. To do this, you need to minimize <em>heat loss</em> while choosing a route that doesn't require the crucible to go in a <em>straight line</em> for too long.</p>
<p>Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.</p>
<p>For example:</p>
<pre><code>2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
</code></pre>
<p>Each city block is marked by a single digit that represents the <em>amount of heat loss if the crucible enters that block</em>. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)</p>
<p>Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move <em>at most three blocks</em> in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.</p>
<p>One way to <em>minimize heat loss</em> is this path:</p>
<pre><code>2<em>&gt;</em><em>&gt;</em>34<em>^</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>1323
32<em>v</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>35<em>v</em>5623
32552456<em>v</em><em>&gt;</em><em>&gt;</em>54
3446585845<em>v</em>52
4546657867<em>v</em><em>&gt;</em>6
14385987984<em>v</em>4
44578769877<em>v</em>6
36378779796<em>v</em><em>&gt;</em>
465496798688<em>v</em>
456467998645<em>v</em>
12246868655<em>&lt;</em><em>v</em>
25465488877<em>v</em>5
43226746555<em>v</em><em>&gt;</em>
</code></pre>
<p>This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only <code><em>102</em></code>.</p>
<p>Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, <em>what is the least heat loss it can incur?</em></p>
</article>

In [41]:
lines = get_test_input_lines(DAY)

In [51]:
lines = get_input_lines(DAY)

In [52]:
vmap = Map.from_object(lines)
for y in range(vmap.h):
    for x in range(vmap.w):
        vmap.map[y][x] = int(vmap.map[y][x])
# vmap.show()

In [53]:
pmap = Map(vmap.h, vmap.w)
for y in range(pmap.h):
    for x in range(pmap.w):
        pmap.map[y][x] = {'r': -1, 'l': -1, 'd': -1, 'u': -1}
pmap.map[0][0] = {'r': 0, 'l': 0, 'd': 0, 'u': 0}

def step(pmap: Map, vmap: Map) -> bool:
    any_change = False
    for y, line in enumerate(pmap.map):
        for x, value_dict in enumerate(line):
            for direction, value in value_dict.items():
                if value == -1:
                    continue
                for dy, dx, new_direction in [
                    (0, 1, 'r'), (0, -1, 'l'), (1, 0, 'd'), (-1, 0, 'u'),
                    (0, 2, 'r'), (0, -2, 'l'), (2, 0, 'd'), (-2, 0, 'u'),
                    (0, 3, 'r'), (0, -3, 'l'), (3, 0, 'd'), (-3, 0, 'u')
                ]:
                    if direction == new_direction:
                        continue
                    if direction == 'r' and new_direction == 'l':
                        continue
                    if direction == 'l' and new_direction == 'r':
                        continue
                    if direction == 'd' and new_direction == 'u':
                        continue
                    if direction == 'u' and new_direction == 'd':
                        continue
                    new_y, new_x = y + dy, x + dx
                    if new_x < 0 or new_x >= pmap.w or new_y < 0 or new_y >= pmap.h:
                        continue
                    new_value_dict = pmap.map[new_y][new_x]
                    new_value = new_value_dict[new_direction]
                    # print(f'{x=}, {y=}, {dx=}, {dy=}, {new_x=}, {new_y=}')
                    if dx == 0:
                        if dy < 0:
                            sum_calc = sum([line[x] for line in vmap.map[y + dy:y]])
                        else:
                            sum_calc = sum([line[x] for line in vmap.map[y + 1:y + dy + 1]])
                    else:
                        if dx < 0:
                            sum_calc = sum(vmap.map[y][x + dx:x])
                        else:
                            sum_calc = sum(vmap.map[y][x + 1:x + dx + 1])

                    new_value_calculated = value + sum_calc

                    if new_value == -1 or new_value_calculated < new_value:
                        # print(f'{x=} {y=} {dx=} {dy=}, {new_x=} {new_y=}, {value} + {sum_calc} = {new_value_calculated}, {new_value}')
                        # print(f'{x=}, {y=}, {dx=}, {dy=}, {new_x=}, {new_y=}, {new_value_calced=}, {new_value_tuple=}')
                        pmap.map[new_y][new_x][new_direction] = new_value_calculated
                        any_change = True
    return any_change

def show(map: Map):
    for line in map.map:
        for value in line:
            value_str = ','.join([f'{v}{k}' for k,v in value.items()])
            print(f'{value_str:20} ', end='')
        print()

i = 0
while step(pmap, vmap):
    i += 1
    # break
print(f'{i=}')

# show(pmap)

i=37


In [54]:
result = min([v for v in pmap.map[-1][-1].values() if v != -1])
print(result)

1110


In [55]:
send_result(DAY, 1, result)

"That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two] (solved in 95:29)"

In [56]:
show_task(DAY, 2)

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The crucibles of lava simply aren't large enough to provide an adequate supply of lava to the machine parts factory. Instead, the Elves are going to upgrade to <em>ultra crucibles</em>.</p>
<p>Ultra crucibles are even more difficult to steer than normal crucibles. Not only do they have trouble going in a straight line, but they also have trouble turning!</p>
<p>Once an ultra crucible starts moving in a direction, it needs to move <em>a minimum of four blocks</em> in that direction before it can turn (or even before it can stop at the end). However, it will eventually start to get wobbly: an ultra crucible can move a maximum of <em>ten consecutive blocks</em> without turning.</p>
<p>In the above example, an ultra crucible could follow this path to minimize heat loss:</p>
<pre><code>2<em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>1323
32154535<em>v</em>5623
32552456<em>v</em>4254
34465858<em>v</em>5452
45466578<em>v</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>
143859879845<em>v</em>
445787698776<em>v</em>
363787797965<em>v</em>
465496798688<em>v</em>
456467998645<em>v</em>
122468686556<em>v</em>
254654888773<em>v</em>
432267465553<em>v</em>
</code></pre>
<p>In the above example, an ultra crucible would incur the minimum possible heat loss of <code><em>94</em></code>.</p>
<p>Here's another example:</p>
<pre><code>111111111111
999999999991
999999999991
999999999991
999999999991
</code></pre>
<p>Sadly, an ultra crucible would need to take an unfortunate path like this one:</p>
<pre><code>1<em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>1111
9999999<em>v</em>9991
9999999<em>v</em>9991
9999999<em>v</em>9991
9999999<em>v</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em>
</code></pre>
<p>This route causes the ultra crucible to incur the minimum possible heat loss of <code><em>71</em></code>.</p>
<p>Directing the <em>ultra crucible</em> from the lava pool to the machine parts factory, <em>what is the least heat loss it can incur?</em></p>
</article>

In [57]:
lines = get_test_input_lines(DAY)

In [64]:
lines = get_input_lines(DAY)

In [65]:
vmap = Map.from_object(lines)
for y in range(vmap.h):
    for x in range(vmap.w):
        vmap.map[y][x] = int(vmap.map[y][x])
# vmap.show()

In [66]:
pmap = Map(vmap.h, vmap.w)
for y in range(pmap.h):
    for x in range(pmap.w):
        pmap.map[y][x] = {'r': -1, 'l': -1, 'd': -1, 'u': -1}
pmap.map[0][0] = {'r': 0, 'l': 0, 'd': 0, 'u': 0}

def step(pmap: Map, vmap: Map) -> bool:
    any_change = False
    for y, line in enumerate(pmap.map):
        for x, value_dict in enumerate(line):
            for direction, value in value_dict.items():
                if value == -1:
                    continue
                for dy, dx, new_direction in [
                    (0, 4, 'r'), (0, -4, 'l'), (4, 0, 'd'), (-4, 0, 'u'),
                    (0, 5, 'r'), (0, -5, 'l'), (5, 0, 'd'), (-5, 0, 'u'),
                    (0, 6, 'r'), (0, -6, 'l'), (6, 0, 'd'), (-6, 0, 'u'),
                    (0, 7, 'r'), (0, -7, 'l'), (7, 0, 'd'), (-7, 0, 'u'),
                    (0, 8, 'r'), (0, -8, 'l'), (8, 0, 'd'), (-8, 0, 'u'),
                    (0, 9, 'r'), (0, -9, 'l'), (9, 0, 'd'), (-9, 0, 'u'),
                    (0, 10, 'r'), (0, -10, 'l'), (10, 0, 'd'), (-10, 0, 'u'),
                ]:
                    if direction == new_direction:
                        continue
                    if direction == 'r' and new_direction == 'l':
                        continue
                    if direction == 'l' and new_direction == 'r':
                        continue
                    if direction == 'd' and new_direction == 'u':
                        continue
                    if direction == 'u' and new_direction == 'd':
                        continue
                    new_y, new_x = y + dy, x + dx
                    if new_x < 0 or new_x >= pmap.w or new_y < 0 or new_y >= pmap.h:
                        continue
                    new_value_dict = pmap.map[new_y][new_x]
                    new_value = new_value_dict[new_direction]
                    if dx == 0:
                        if dy < 0:
                            sum_calc = sum([line[x] for line in vmap.map[y + dy:y]])
                        else:
                            sum_calc = sum([line[x] for line in vmap.map[y + 1:y + dy + 1]])
                    else:
                        if dx < 0:
                            sum_calc = sum(vmap.map[y][x + dx:x])
                        else:
                            sum_calc = sum(vmap.map[y][x + 1:x + dx + 1])

                    new_value_calculated = value + sum_calc

                    if new_value == -1 or new_value_calculated < new_value:
                        pmap.map[new_y][new_x][new_direction] = new_value_calculated
                        any_change = True
    return any_change

def show(map: Map):
    for line in map.map:
        for value in line:
            value_str = ','.join([f'{v}{k}' for k,v in value.items()])
            print(f'{value_str:20} ', end='')
        print()

i = 0
while step(pmap, vmap):
    i += 1
    # break
print(f'{i=}')

# show(pmap)

i=11


In [67]:
result = min([v for v in pmap.map[-1][-1].values() if v != -1])
print(result)

1294


In [68]:
send_result(DAY, 2, result)

"That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 17! (solved in 05:27)"